In [1]:
# pip install seaborn

In [2]:
%matplotlib qt5
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as scyint
import pandas as pd
from mathphys import constants
from mathphys.functions import save_pickle, load_pickle
from mathphys.beam_optics import beam_rigidity


In [3]:
import seaborn

In [4]:
import touschek_pack.functions as tousfunc

In [5]:
import touschek_pack.Classes as tousclass

In [6]:
# eu preciso lembrar de depois acrescentar uma função que faça o trakcing nao tornando explicito a diferenciação entre desvios de energia 
# positivos e negativos para contabilizar de maneira correta a taxa de espalhamento touschek porque na biblioteca lifetime a taxa de espalhamento
# calculada esta implementada com base nos espalhamentos tanto negativos como positivos

In [7]:
# This function will probably will be in my repositories
# I dont know if I will use it again, but it seems to me that it could be uselfull in some point

def extract_delt(groups, deltas):
    c = 0
    big_list = []
    for lists in groups:
        lil_list = []
        for _ in lists:
            lil_list.append(c)
            c+=1
            
        big_list.append(lil_list)
    
    sep_deltas = []
    comp_l = []

    for iten in big_list:
        sep_deltas.append(deltas[iten])
        comp_l.append(len(iten))


    return sep_deltas, comp_l

In [8]:
acc = pymodels.si.create_accelerator()
acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(acc)

tousan = tousclass.Tous_analysis(acc)
spos = pyaccel.lattice.find_spos(acc, indices='closed')


In [85]:
indices = pyaccel.lattice.find_indices(acc, 'fam_name', 'B1')

In [87]:
s = spos[29:51]
dic = tousan.get_table(s)

In [121]:
df = pd.DataFrame(dic)

df = df.set_index('lost_positions')

# Crie um mapa de calor com o matplotlib
plt.figure(figsize=(10, 6))  # Define o tamanho da figura
heatmap = plt.pcolor(df, cmap='jet')  # Crie o mapa de calor
plt.colorbar(heatmap)  # Adicione uma barra de cores
plt.xticks(np.arange(df.shape[1]) + 0.5, df.columns, rotation=90)  #  Rótulos do eixo X
plt.yticks(np.arange(df.shape[0]) + 0.5, df.index)  # Rótulos do eixo Y
plt.title('Mapa de Calor')  # Define o título do gráfico
plt.show()  # Mostra o mapa de calor

In [ ]:
ltime = pyaccel.lifetime.Lifetime(acc)
tous_rate = ltime.touschek_data['rate']
np.where(tous_rate <= 1e-6)[0].shape

In [106]:
df.index

Float64Index([ 73.37,  91.24,  91.95,  92.07,  95.78,  99.29,  99.36,  99.44,
               99.53,  99.73,
              ...
               18.02,  18.22,  43.77, 410.17,   17.7,  21.53,  21.77,  43.62,
               43.94,  24.92],
             dtype='float64', name='lost_positions', length=129)